# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import textwrap
import logging
import glob

from tqdm import tqdm
#from matplotlib import pyplot
#from skopt import BayesSearchCV
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid, train_test_split, ParameterSampler
#from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_squared_log_error, r2_score


from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope

#from keras.models import Sequential
#from keras.layers import LSTM, Dense, Dropout, Input
#from keras.callbacks import EarlyStopping

In [2]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logging.log"),
        logging.StreamHandler()
    ]
)

# Load - Train - Save Models without HP

In [3]:
# === SETTINGS ===
base_path = "../../07_Imputation/CSV/exports/CIR-16/impute/"
observation_window = 'o4'
label = 'los'
model_output_dir = "models/"
plot_dir = "plots/03_Error_Metric_Plots"

os.makedirs(model_output_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

In [4]:
# === HELPER FUNCTION TO MATCH FILES ===
def find_file(path, pattern):
    matches = glob.glob(os.path.join(path, pattern))
    return matches[0] if matches else None

In [5]:
def plot_error_metrics(y_true, y_pred, file_prefix: str, plot_label: str, config_label: str = "no HP"):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred) * 100

    error_metrics = ['MSE', 'MAE', 'RMSE']
    values = [mse, mae, rmse]

    msle = np.nan
    try:
        msle = mean_squared_log_error(y_true, y_pred)
        logging.info(f"{plot_label.title()} Set MSLE: {msle:.4f}")
        error_metrics.append('MSLE')
        values.append(msle)
    except ValueError:
        logging.info(f"{plot_label.title()} Set MSLE: Not computable due to negative values.")

    # Bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
    plt.xlabel('Error Metric')
    plt.ylabel('Value')
    plt.title(f'Error Metrics ({plot_label.title()} Set) - {config_label}')
    plt.savefig(f"{plot_dir}/{file_prefix}_{plot_label}_{config_label.replace(' ', '_')}_error_metrics.png", dpi=300, bbox_inches='tight')
    plt.close()

    # R² pie plot
    plt.figure(figsize=(6, 6))
    if r2 >= 0:
        plt.pie([r2, 100 - r2], labels=['Explained Variance (R2)', 'Unexplained Variance'],
                colors=['lightblue', 'lightgrey'], autopct='%1.1f%%')
    else:
        plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')
    plt.title(f'Explained Variance by R² ({plot_label.title()} Set) - {config_label}')
    plt.savefig(f"{plot_dir}/{file_prefix}_{plot_label}_{config_label.replace(' ', '_')}_R2.png", dpi=300, bbox_inches='tight')
    plt.close()

    return {
        "MSE": mse,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2,
        "MSLE": msle
    }


In [6]:
# === MAIN LOOP ===
all_metrics = []  # collect all results for summary pivot plot
seq_folders = sorted([f for f in os.listdir(base_path) if f.startswith("seq_")])

for folder in seq_folders:
    logging.info(f"Processing folder: {folder}")
    load_path = os.path.join(base_path, folder)
    load_path_label = os.path.join(base_path, "labels")

    try:
        # Load data
        X_train = pd.read_csv(find_file(load_path, f"{observation_window}_X_train*.csv"))
        y_train = pd.read_csv(find_file(load_path_label, f"{observation_window}_y_train_{label}.csv"))

        X_validate = pd.read_csv(find_file(load_path, f"{observation_window}_X_validate*.csv"))
        y_validate = pd.read_csv(find_file(load_path_label, f"{observation_window}_y_validate_{label}.csv"))

        X_test = pd.read_csv(find_file(load_path, f"{observation_window}_X_test*.csv"))
        y_test = pd.read_csv(find_file(load_path_label, f"{observation_window}_y_test_{label}.csv"))

        X_external = pd.read_csv(find_file(load_path, f"{observation_window}_X_external*.csv"))
        y_external = pd.read_csv(find_file(load_path_label, f"{observation_window}_y_external_{label}.csv"))

        # Train model
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)

        # Save model
        file_prefix = f"{folder}_{observation_window}_{label}"
        model.save_model(os.path.join(model_output_dir, f"{file_prefix}_model.json"))
        logging.info(f"Saved model to {model_output_dir}/{file_prefix}_model.json")

        # Predict
        y_pred_test = model.predict(X_test)
        y_pred_ext = model.predict(X_external)

        # Log basic metrics
        mse_test = mean_squared_error(y_test, y_pred_test)
        mae_test = mean_absolute_error(y_test, y_pred_test)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, y_pred_test) * 100

        mse_ext = mean_squared_error(y_external, y_pred_ext)
        mae_ext = mean_absolute_error(y_external, y_pred_ext)
        rmse_ext = np.sqrt(mse_ext)
        r2_ext = r2_score(y_external, y_pred_ext) * 100

        logging.info(f"[{folder}] Test MSE: {mse_test:.4f} | MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f} | R2: {r2_test:.2f}")
        logging.info(f"[{folder}] External MSE: {mse_ext:.4f} | MAE: {mae_ext:.4f} | RMSE: {rmse_ext:.4f} | R2: {r2_ext:.2f}")

        # Generate plots and capture metrics
        internal_metrics = plot_error_metrics(y_test, y_pred_test, file_prefix, plot_label='internal', config_label="no HP")
        external_metrics = plot_error_metrics(y_external, y_pred_ext, file_prefix, plot_label='external', config_label="no HP")

        # Append to all_metrics
        all_metrics.append({"folder": folder, "dataset": "internal", **internal_metrics})
        all_metrics.append({"folder": folder, "dataset": "external", **external_metrics})

    except Exception as e:
        logging.error(f"Failed in folder {folder}: {str(e)}")

# === AFTER MAIN LOOP ===
metrics_df = pd.DataFrame(all_metrics)

# Save metrics to CSV
summary_csv_path = "plots/03_Error_Metric_Plots/all_seq_metrics.csv"
metrics_df.to_csv(summary_csv_path, index=False)
logging.info(f"Saved summary metrics to: {summary_csv_path}")


# Melt the dataframe for plotting
metrics_melted = metrics_df.melt(
    id_vars=["folder", "dataset"],
    value_vars=["MSE", "MAE", "RMSE", "R2", "MSLE"],
    var_name="Metric",
    value_name="Value"
)

# Drop missing MSLE rows (optional)
metrics_melted = metrics_melted.dropna()

# === CREATE ONE PLOT PER METRIC ===
unique_metrics = metrics_melted["Metric"].unique()

for metric in unique_metrics:
    plt.figure(figsize=(12, 6))
    subset = metrics_melted[metrics_melted["Metric"] == metric]
    
    sns.barplot(data=subset, x="folder", y="Value", hue="dataset", palette="Set2", errorbar=None)
    plt.title(f"{metric} Comparison (Internal vs External)")
    plt.ylabel(metric)
    plt.xlabel("Sequence Folder")
    plt.xticks(rotation=45)
    plt.legend(title="Dataset")
    plt.tight_layout()
    
    metric_plot_path = f"plots/03_Error_Metric_Plots/metric_{metric}_comparison_plot.png"
    plt.savefig(metric_plot_path, dpi=300)
    plt.close()

2025-05-16 00:14:54,439 - INFO - Processing folder: seq_00
2025-05-16 00:15:11,089 - INFO - Saved model to models//seq_00_o4_los_model.json
2025-05-16 00:15:11,574 - INFO - [seq_00] Test MSE: 4.8175 | MAE: 1.6727 | RMSE: 2.1949 | R2: 4.61
2025-05-16 00:15:11,574 - INFO - [seq_00] External MSE: 7.1572 | MAE: 2.0611 | RMSE: 2.6753 | R2: -63.30
2025-05-16 00:15:11,592 - INFO - Internal Set MSLE: 0.2984
2025-05-16 00:15:12,906 - INFO - External Set MSLE: 0.4795
2025-05-16 00:15:14,074 - INFO - Processing folder: seq_01
2025-05-16 00:15:30,287 - INFO - Saved model to models//seq_01_o4_los_model.json
2025-05-16 00:15:30,757 - INFO - [seq_01] Test MSE: 4.7063 | MAE: 1.7996 | RMSE: 2.1694 | R2: 6.81
2025-05-16 00:15:30,759 - INFO - [seq_01] External MSE: 8.2710 | MAE: 2.4191 | RMSE: 2.8759 | R2: -88.72
2025-05-16 00:15:30,773 - INFO - Internal Set MSLE: 0.3011
2025-05-16 00:15:32,195 - INFO - External Set MSLE: 0.5460
2025-05-16 00:15:33,349 - INFO - Processing folder: seq_02
2025-05-16 00:15:

In [ ]:
# Base path and settings
base_path = "../../07_Imputation/CSV/exports/CIR-16/impute/"
observation_window = 'o4'
label = 'los'
model_output_dir = "models/"
os.makedirs(model_output_dir, exist_ok=True)

# Get all seq_* folders
seq_folders = sorted([f for f in os.listdir(base_path) if f.startswith("seq_")])

# Helper to match files ignoring rank suffix
def find_file(path, pattern):
    matches = glob.glob(os.path.join(path, pattern))
    return matches[0] if matches else None

for folder in seq_folders:
    logging.info(f"Processing folder: {folder}")
    load_path = os.path.join(base_path, folder)

    try:
        # Dynamically find files
        X_train_path = find_file(load_path, f"{observation_window}_X_train*.csv")
        y_train_path = find_file(load_path, f"{observation_window}_y_train_{label}.csv")

        X_validate_path = find_file(load_path, f"{observation_window}_X_validate*.csv")
        y_validate_path = find_file(load_path, f"{observation_window}_y_validate_{label}.csv")

        X_test_path = find_file(load_path, f"{observation_window}_X_test*.csv")
        y_test_path = find_file(load_path, f"{observation_window}_y_test_{label}.csv")

        X_external_path = find_file(load_path, f"{observation_window}_X_external*.csv")
        y_external_path = find_file(load_path, f"{observation_window}_y_external_{label}.csv")

        # Validate presence
        if not all([X_train_path, y_train_path, X_validate_path, y_validate_path, X_test_path, y_test_path, X_external_path, y_external_path]):
            raise FileNotFoundError("One or more required files not found in folder.")

        # Load data
        X_train = pd.read_csv(X_train_path)
        y_train = pd.read_csv(y_train_path)

        X_validate = pd.read_csv(X_validate_path)
        y_validate = pd.read_csv(y_validate_path)

        X_test = pd.read_csv(X_test_path)
        y_test = pd.read_csv(y_test_path)

        X_external = pd.read_csv(X_external_path)
        y_external = pd.read_csv(y_external_path)

        # Train model
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)

        # Save model
        file_name = f"{folder}_{observation_window}_{label}_model.json"
        file_path = os.path.join(model_output_dir, file_name)
        model.save_model(file_path)
        logging.info(f"Saved model to {file_path}")

        # Evaluate on test
        y_pred_test = model.predict(X_test)
        mse_test = mean_squared_error(y_test, y_pred_test)
        mae_test = mean_absolute_error(y_test, y_pred_test)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, y_pred_test) * 100

        # Evaluate on external
        y_pred_ext = model.predict(X_external)
        mse_ext = mean_squared_error(y_external, y_pred_ext)
        mae_ext = mean_absolute_error(y_external, y_pred_ext)
        rmse_ext = np.sqrt(mse_ext)
        r2_ext = r2_score(y_external, y_pred_ext) * 100

        # Log metrics
        logging.info(f"[{folder}] Test MSE: {mse_test:.4f} | MAE: {mae_test:.4f} | RMSE: {rmse_test:.4f} | R2: {r2_test:.2f}")
        logging.info(f"[{folder}] External MSE: {mse_ext:.4f} | MAE: {mae_ext:.4f} | RMSE: {rmse_ext:.4f} | R2: {r2_ext:.2f}")

        file_prefix = f"{folder}_{observation_window}_{label}"

        # For test set
        plot_error_metrics(y_test, y_pred_test, file_prefix, plot_label='internal')

        # For external validation set
        plot_error_metrics(y_external, y_pred_ext, file_prefix, plot_label='external')


    except Exception as e:
        logging.error(f"Failed in folder {folder}: {str(e)}")


# Plot
## Error Metrics

In [ ]:
"""
Plots error metrics (MSE, MAE, RMSE, MSLE if possible) and R² for a prediction set.
    
Args:
    y_true (pd.Series or np.array): Ground truth values.
    y_pred (np.array): Predicted values from the model.
    file_prefix (str): Prefix for output file naming (e.g., 'seq_00_o4_los').
    plot_label (str): 'internal' or 'external' or any identifier to mark the dataset.
"""


def plot_error_metrics(y_true, y_pred, file_prefix: str, plot_label: str):
    
    # Metrics calculation
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred) * 100

    print(f"{plot_label.title()} Set MSE: {mse:.4f}")
    print(f"{plot_label.title()} Set MAE: {mae:.4f}")
    print(f"{plot_label.title()} Set RMSE: {rmse:.4f}")
    print(f"{plot_label.title()} Set R2: {r2:.4f}")

    # Prepare metrics for bar plot
    error_metrics = ['MSE', 'MAE', 'RMSE']
    values = [mse, mae, rmse]

    # Optional MSLE
    try:
        msle = mean_squared_log_error(y_true, y_pred)
        print(f"{plot_label.title()} Set MSLE: {msle:.4f}")
        error_metrics.append('MSLE')
        values.append(msle)
    except ValueError:
        print(f"{plot_label.title()} Set MSLE: Not computable due to negative values.")

    # Create output folder
    plot_dir = "plots/03_Error_Metric_Plots"
    os.makedirs(plot_dir, exist_ok=True)

    # Bar plot of error metrics
    plt.figure(figsize=(10, 6))
    plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
    plt.xlabel('Error Metric')
    plt.ylabel('Value')
    plt.title(f'Comparison of Error Metrics ({plot_label.title()} Set)')
    plt.savefig(f"{plot_dir}/{file_prefix}_{plot_label}_error_metrics.png", dpi=300, bbox_inches='tight')
    plt.show()

    # Pie chart for R²
    plt.figure(figsize=(6, 6))
    if r2 >= 0:
        plt.pie([r2, 100 - r2],
                labels=['Explained Variance (R2)', 'Unexplained Variance'],
                colors=['lightblue', 'lightgrey'],
                autopct='%1.1f%%')
    else:
        plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')

    plt.title(f'{plot_label.title()} Set Explained Variance by R²')
    plt.savefig(f"{plot_dir}/{file_prefix}_{plot_label}_R2.png", dpi=300, bbox_inches='tight')
    plt.show()


# Train Model without HP

In [ ]:
# Default XGBoost Model
model = xgb.XGBRegressor(objective='reg:squarederror')

model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Predict on the external validation set (eICU data)
y_pred_external = model.predict(X_external)

# HP GridSearchCV
## To slow

In [ ]:
"""
A smaller learning rate makes the boosting
process more robust and can lead to better
generalization but requires more trees
(higher n_estimators) to achieve the same result.
A larger learning rate speeds up training bu
may risk overfitting.
"""

# Define the parameter grid

param_grid = {
    'n_estimators': [100, 200, 300], # controls the total number of trees in the ensemble
    'learning_rate': np.arange(0.01, 1.02, 0.2),
    'max_depth': np.arange(1, 10, 1),
    'reg_lambda': np.arange(0.1, 15.1, 1),
    'reg_alpha': np.arange(0.1, 15.1, 1)
}

# Create an XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Create GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', 
                           cv=3,  # Number of folds for cross-validation
                           verbose=1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)

# Predict on the validation set with the best model
y_pred_validate = grid_search.predict(X_validate)

# Optionally: Evaluate the model on the validation set
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_validate, y_pred_validate)
print("Validation MSE:", mse)

# HP RandomizedSearchCV & Train Model
Choose randomly samples a subset of hyperparameter combinations

In [ ]:
# Define the parameter grid
param_distributions = {
    'n_estimators': [100, 200, 300],
    'learning_rate': np.arange(0.01, 1.02, 0.2),
    'max_depth': np.arange(1, 10, 1),
    'reg_lambda': np.arange(0.1, 15.1, 1),
    'reg_alpha': np.arange(0.1, 15.1, 1)
}

# Number of random samples
n_iter = 50

# Generate random combinations
param_list = list(ParameterSampler(param_distributions, n_iter=n_iter, random_state=42))

# Tracking best model
best_score = float('inf')
best_params = None
best_model = None

# Progress bar
for params in tqdm(param_list, desc="Hyperparameter tuning"):
    model = xgb.XGBRegressor(objective='reg:squarederror', **params)
    model.fit(X_train, y_train)
    
    # Predict on validation set
    y_pred_val = model.predict(X_validate)
    
    # Evaluate with MSE
    mse = mean_squared_error(y_validate, y_pred_val)
    
    if mse < best_score:
        best_score = mse
        best_params = params
        best_model = model

# Evaluate best model on test set
y_pred = best_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)

# Evaluate on external validation set
y_pred_external = best_model.predict(X_external)
mse_external = mean_squared_error(y_external, y_pred_external)
mae_external = mean_absolute_error(y_external, y_pred_external)

# Results
logging.info(f"Best parameters: {best_params}")
logging.info(f"Best validation MSE: {best_score}")
logging.info(f"Test Set - MSE: {mse_test}, MAE: {mae_test}")
logging.info(f"External Validation Set - MSE: {mse_external}, MAE: {mae_external}")

# HP Bayesian Optimization & Train Model

In [ ]:
# Initialize tqdm progress bar
pbar = tqdm(total=50, desc="Bayesian Optimization Progress")

# Callback to update tqdm
def on_step(optim_result):
    pbar.update(1)

# Define the parameter search space
param_space = {
    'n_estimators': (100, 300),
    'learning_rate': (0.01, 1.0, 'log-uniform'),
    'max_depth': (1, 10),
    'reg_lambda': (0.1, 15.0),
    'reg_alpha': (0.1, 15.0)
}

# Create the XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Create BayesSearchCV for Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=param_space,
    scoring='neg_mean_squared_error',
    n_iter=50,
    cv=3,
    verbose=0,
    random_state=42
)

# Fit BayesSearchCV with tqdm callback
bayes_search.fit(X_train, y_train, callback=on_step)
pbar.close()

# Log best parameters and score
logging.info("Best parameters: %s", bayes_search.best_params_)
logging.info("Best score (negative MSE): %.4f", bayes_search.best_score_)

# Predict on the validation set with the best model
y_pred_validate = bayes_search.predict(X_validate)

# Evaluate the model on the validation set
mse_validate = mean_squared_error(y_validate, y_pred_validate)
mae_validate = mean_absolute_error(y_validate, y_pred_validate)
logging.info("Validation MSE: %.4f", mse_validate)
logging.info("Validation MAE: %.4f", mae_validate)

# Extract the best hyperparameters from BayesSearchCV
best_params = bayes_search.best_params_

# Initialize the XGBoost model with the best hyperparameters
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    reg_lambda=best_params['reg_lambda'],
    reg_alpha=best_params['reg_alpha']
)

# Train the model on the training set
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Predict on the external validation set (eICU data)
y_pred_external = model.predict(X_external)

# Evaluate the model on the test set
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)

# Evaluate the model on the external validation set
mse_external = mean_squared_error(y_external, y_pred_external)
mae_external = mean_absolute_error(y_external, y_pred_external)

# Log final evaluation metrics
logging.info("Test Set - MSE: %.4f, MAE: %.4f", mse_test, mae_test)
logging.info("External Validation Set (eICU) - MSE: %.4f, MAE: %.4f", mse_external, mae_external)

# HP HyperOpt & Train Model

In [ ]:
# Define the number of evaluations
MAX_EVALS = 50

# Initialize tqdm progress bar
pbar = tqdm(total=MAX_EVALS, desc="HyperOpt Progress")

# Define the wrapped objective function
def objective(params):
    model = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=int(params['n_estimators']),
        learning_rate=params['learning_rate'],
        max_depth=int(params['max_depth']),
        reg_lambda=params['reg_lambda'],
        reg_alpha=params['reg_alpha']
    )
    
    # Fit the model
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    y_pred_validate = model.predict(X_validate)
    
    # Compute the MSE
    mse = mean_squared_error(y_validate, y_pred_validate)

    # Log the result
    logging.info("Params: %s | Validation MSE: %.4f", params, mse)
    
    # Update progress bar
    pbar.update(1)

    return {'loss': mse, 'status': 'ok'}

# Define the parameter search space
param_space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 100, 300, 50)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(1.0)),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 10, 1)),
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 15.0),
    'reg_alpha': hp.uniform('reg_alpha', 0.1, 15.0)
}

# Create a Trials object to keep track of the search
trials = Trials()

# Perform the hyperparameter search
best = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=MAX_EVALS,
    trials=trials,
    show_progressbar=False  # Disable internal bar to avoid overlap with tqdm
)

# Close progress bar
pbar.close()

# Log the best parameters
logging.info("Best parameters: %s", best)

# Initialize the XGBoost model with the best hyperparameters
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=int(best['n_estimators']),
    learning_rate=best['learning_rate'],
    max_depth=int(best['max_depth']),
    reg_lambda=best['reg_lambda'],
    reg_alpha=best['reg_alpha']
)

# Train the model on the training set
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Predict on the external validation set (eICU data)
y_pred_external = model.predict(X_external)

# Evaluate the model on the test set
mse_test = mean_squared_error(y_test, y_pred)
mae_test = mean_absolute_error(y_test, y_pred)

# Evaluate the model on the external validation set
mse_external = mean_squared_error(y_external, y_pred_external)
mae_external = mean_absolute_error(y_external, y_pred_external)

# Log final evaluation results
logging.info("Test Set - MSE: %.4f, MAE: %.4f", mse_test, mae_test)
logging.info("External Validation Set (eICU) - MSE: %.4f, MAE: %.4f", mse_external, mae_external)

In [ ]:
# Path
save_path = 'CSV/exports/impute/o03_Interpolation/'

# Check if the directory exists, and if not, create it
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save external validation set from eICU
X_external.to_csv(save_path + 'X_external.csv', index=False)
y_external.to_csv(save_path + 'y_external.csv', index=False)

# Save training, validation, and test sets
X_train.to_csv(save_path + 'X_train.csv', index=False)
y_train.to_csv(save_path + 'y_train.csv', index=False)

X_validate.to_csv(save_path + 'X_validate.csv', index=False)
y_validate.to_csv(save_path + 'y_validate.csv', index=False)

X_test.to_csv(save_path + 'X_test.csv', index=False)
y_test.to_csv(save_path + 'y_test.csv', index=False)

# Save Model

In [ ]:
# Define the directory and file path

name = f"{file_name}_model.json"
directory = 'models/'

file_path = os.path.join(directory, name)

# Create the directory if it does not exist
os.makedirs(directory, exist_ok=True)

# Save the model as a JSON file
model.save_model(file_path)

# Load Model

In [ ]:
# Define the model file path
file_name = "06"  # replace with the actual name you used before saving
directory = 'models/'
file_path = os.path.join(directory, f"{file_name}_model.json")

# Load the model
model = xgb.XGBRegressor()
model.load_model(file_path)

# Predict on the test set
y_pred = model.predict(X_test)

# Predict on the external validation set
y_pred_external = model.predict(X_external)

# Test Set Plots

In [ ]:
# Metrics calculation
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred) * 100

print(f"Test Set MSE: {mse:.4f}")
print(f"Test Set MAE: {mae:.4f}")
print(f"Test Set RMSE: {rmse:.4f}")
print(f"Test Set R2: {r2:.4f}")

# Initialize error metrics
error_metrics = ['MSE', 'MAE', 'RMSE']
values = [mse, mae, rmse]

# Try to calculate MSLE
try:
    msle = mean_squared_log_error(y_test, y_pred)
    print(f"Test Set MSLE: {msle:.4f}")
    
    # Add MSLE to the list of metrics if applicable
    error_metrics.append('MSLE')
    values.append(msle)
except ValueError:
    print("Mean Squared Logarithmic Error cannot be calculated because targets contain negative values.")

# Plot error metrics (with or without MSLE)
plt.figure(figsize=(10, 6))
plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
plt.xlabel('Error Metric')
plt.ylabel('Value')
plt.title('Comparison of Error Metrics')

output_path = f"plots/03_Error_Metric_Plots/{file_name}_internal_error_metrics.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# Plotting R-squared (R2) for the test set
plt.figure(figsize=(6, 6))

if r2 >= 0:
    plt.pie([r2, 100 - r2], 
            labels=['Explained Variance (R2)', 'Unexplained Variance'], 
            colors=['lightblue', 'lightgrey'], autopct='%1.1f%%')
else:
    plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')

plt.title('Test Set Explained Variance by R-squared (R2)')
output_path = f"plots/03_Error_Metric_Plots/{file_name}_internal_R2.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# External Validation Plots

In [ ]:
# Predict on the external validation set (eICU data)
y_pred_external = model.predict(X_external)

# Metrics for external validation set
mse_external = mean_squared_error(y_external, y_pred_external)
mae_external = mean_absolute_error(y_external, y_pred_external)
rmse_external = np.sqrt(mse_external)
r2_external = r2_score(y_external, y_pred_external) * 100

print(f"External Validation Set MSE: {mse_external:.4f}")
print(f"External Validation Set MAE: {mae_external:.4f}")
print(f"External Validation Set RMSE: {rmse_external:.4f}")
print(f"External Validation Set R2: {r2_external:.4f}")

# Initialize error metrics
error_metrics = ['MSE', 'MAE', 'RMSE']
values = [mse_external, mae_external, rmse_external]

# Try to calculate MSLE
try:
    msle_external = mean_squared_log_error(y_external, y_pred_external)
    print(f"External Validation Set MSLE: {msle_external:.4f}")
    
    # Add MSLE to the list of metrics if applicable
    error_metrics.append('MSLE')
    values.append(msle_external)
except ValueError:
    print("Mean Squared Logarithmic Error cannot be calculated because targets contain negative values.")

# Plot error metrics (with or without MSLE)
plt.figure(figsize=(10, 6))
plt.bar(error_metrics, values, color=['blue', 'green', 'red', 'orange'][:len(error_metrics)])
plt.xlabel('Error Metric')
plt.ylabel('Value')
plt.title('Comparison of Error Metrics')


output_path = f"plots/03_Error_Metric_Plots/{file_name}_external_error_metrics.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# Plotting R-squared (R2) for the external validation set
plt.figure(figsize=(6, 6))

if r2_external >= 0:
    plt.pie([r2_external, 100 - r2_external], 
            labels=['Explained Variance (R2)', 'Unexplained Variance'], 
            colors=['lightblue', 'lightgrey'], autopct='%1.1f%%')
else:
    plt.pie([100], labels=['Unexplained Variance'], colors=['lightgrey'], autopct='%1.1f%%')

plt.title('Validation Set Explained Variance by R-squared (R2)')

output_path = f"plots/03_Error_Metric_Plots/{file_name}_external_R2.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# Calibration Plot

In [ ]:
# Function to create calibration plot
def plot_calibration(y_true, y_pred, title, filename):
    plt.figure(figsize=(8, 6))
    sns.regplot(x=y_true, y=y_pred, lowess=True, line_kws={'color': 'red'}, scatter_kws={'alpha': 0.4})
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'k--', lw=2)
    plt.xlabel('Actual LOS')
    plt.ylabel('Predicted LOS')
    plt.title(f'Calibration Plot: {title}')
    plt.grid(True)

    output_path = f"plots/04_Calibration_Plots/{filename}.png"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.show()

# Plot for test set
plot_calibration(y_test, y_pred, "Internal Test Set", f"{file_name}_calibration_internal")

# Plot for external set
plot_calibration(y_external, y_pred_external, "External Validation Set", f"{file_name}_calibration_external")

# Most important features

In [ ]:
# Get feature importances
most_important_df = model.feature_importances_

# Create a DataFrame to store feature importances along with their corresponding names
most_important_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': most_important_df})

# Sort the DataFrame by feature importance in descending order
most_important_df = most_important_df.sort_values(by='Importance', ascending=False)

# Scale the importance
most_important_df['Importance'] *= 100000

# Print the top N most important features
top_n = 20  # set features number
print(f"Top {top_n} most important features:")
print(most_important_df.head(top_n))

In [ ]:
top_20_features = most_important_df.head(20).copy()  # Create a copy explicitly
top_20_features['Feature'] = top_20_features['Feature'].apply(lambda x: '\n'.join(textwrap.wrap(x, width=20)))

# Set seaborn style and remove gridlines
sns.set_style("whitegrid")

# Top 20 most important features (create a copy)
top_20_features = most_important_df.head(20).copy()

# Wrap long feature names
top_20_features['Feature'] = top_20_features['Feature'].apply(lambda x: '\n'.join(textwrap.wrap(x, width=20)))

# Plotting
plt.figure(figsize=(18, 11))  # Increase figure size for better visibility
plot = sns.barplot(x='Importance', y='Feature', data=top_20_features, hue='Feature', palette="Blues", legend=False)

# Set font size for labels and title
plt.xlabel('Importance', fontsize=16)
plt.ylabel('Feature', fontsize=16)
plt.title('Top 20 Features with Highest Importance', fontsize=20)

# Save the plot in high resolution
output_path = f"plots/01_Most_Important_SHAP/{file_name}_most_important_features.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# Shap plot

In [ ]:
# Explain the model's predictions using SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# Mean absolute SHAP values to rank feature importance
mean_abs_shap = np.abs(shap_values).mean(axis=0)
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Mean Absolute SHAP Value': mean_abs_shap
}).sort_values(by='Mean Absolute SHAP Value', ascending=False)

# Display the top N most important features
top_n = 20
print(f"Top {top_n} features by SHAP importance:")
print(feature_importance_df.head(top_n))

# Create a Matplotlib figure
plt.figure()

# SHAP Summary Plot with Bee Swarm (distribution of feature impacts)
shap.summary_plot(shap_values, X_train, plot_type="dot", show=False)

# Add grid to the plot
plt.grid(True)

# Save the plot as a PNG image
output_path = f"plots/01_Most_Important_SHAP/{file_name}_shap_plot.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

# Create Shap by saved model

In [ ]:
# Load the XGBoost model from the JSON file
model = xgb.Booster()  # Initialize the Booster object
model.load_model("models\\02_Mean_Impute\\o11_Mean_Scale_Norm_Bayes_.json")

# I must load training - test set

# Create a SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values
shap_values = explainer.shap_values(X_train)

# Generate SHAP summary plot
shap.summary_plot(shap_values, X_train, plot_type="dot", show=False)

# Add grid to the plot
plt.grid(True)

# If i want to save the plot I must change in line 14 the show=True to False
# plt.savefig("shap_summary_plot.png", dpi=300, bbox_inches='tight')

# Display the plot
plt.show()

# Plotting True vs. Predicted LOS

In [ ]:
# Test Set Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', label='Prediction')

# Line for Perfect Prediction
perfect_line = np.linspace(y_test.min(), y_test.max(), 100)
plt.plot(perfect_line, perfect_line, color='red', linestyle='--', label='Perfect Prediction')

# Labels, legend, and grid
plt.xlabel('True LOS')
plt.ylabel('Predicted LOS')
plt.legend()
plt.grid(True)
plt.title('Predicted vs. True LOS (Test Set)')

# Save the plot as a PNG image
output_path = f"plots/02_Prediction_Plot/02_true_vs_pred/{file_name}_true_vs_pred_test_plot.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

# External Validation Set Plot
plt.figure(figsize=(8, 6))
plt.scatter(y_external, y_pred_external, color='blue', label='Prediction')

# Line for Perfect Prediction (y = x)
perfect_line_ext = np.linspace(y_external.min(), y_external.max(), 100)
plt.plot(perfect_line_ext, perfect_line_ext, color='red', linestyle='--', label='Perfect Prediction')

# Labels, legend, and grid
plt.xlabel('True LOS')
plt.ylabel('Predicted LOS')
plt.legend()
plt.grid(True)
plt.title('Predicted vs. True LOS (External Validation Set)')

# Save the plot as a PNG image
output_path = f"plots/02_Prediction_Plot/02_true_vs_pred/{file_name}_true_vs_pred_external_plot.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Convert y_test to a 1D numpy array
y_test = y_test.values.flatten()

# Calculate residuals
residuals = y_test - y_pred

# Plot residuals
plt.figure(figsize=(8, 6))
plt.scatter(y_test, residuals, color='blue', alpha=0.5, label="Residuals")
plt.axhline(y=0, color='red', linestyle='--', label="Zero Line")
plt.axhline(y=mae, color='green', linestyle='--', label=f"MAE = {mae:.2f}")
plt.axhline(y=-mae, color='green', linestyle='--')
plt.xlabel('True LOS')
plt.ylabel('Residuals (True - Predicted)')
plt.title('Residuals Plot with MAE Bounds')
plt.grid(True)

# Place the legend outside of the plot
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))

# Save the plot as a PNG image
output_path = f"plots/02_Prediction_Plot/01_residuals/{file_name}_residuals_plot.png"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("y_test shape:", y_test.shape)
print("y_pred shape:", y_pred.shape)

In [ ]:
print (y_pred)

In [ ]:
# Convert y_test to a 1D numpy array
y_test = y_test.values.flatten()